
<center><h1>Meet your new teacher ... Gemma-Lora Pefting</h1><img src="https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjopwHAmKL3wi6SrOSr1-UDGnxWM1-Qu0kFkEoVK_Wa8570PZBOu4r8e5M6wJfOrjDLUn95VcuqujoIciiGIZk6G0VqNLSDjM_-_7I9Jcj9IwvC1E2FxquO-JKzYGd8gRfgnEzEJVBma3Bd/s1600/Bad+Teacher.jpg" ></center>This code snippet orchestrates a fascinating fusion of technologies for automated essay scoring. <br><br>

**The commented out code is the LoRa training. So, basically I have trained the model with the given dataset, saved it and load the saved model for submission (since the internet must be off).**<br><br>
Let's break it down:

## Imports: 
The code begins by importing necessary libraries. peft and accelerate are used for performance optimization, transformers for working with pre-trained models, and torch for tensor computation. Additionally, it imports modules like bitsandbytes, pandas, datasets, and trl.

## Configuration Setup: 
It initializes configuration settings, including quantization configuration, model path, tokenizer, and model for auto-regressive language modeling (LM). The model is equipped with LoraConfig for Low-Rank Adaptive Attention.

## Model Parameter Analysis: 
The function print_number_of_trainable_model_parameters computes and prints the number of trainable model parameters, which is essential for understanding model complexity and training feasibility.

## Data Preparation: 
It loads a dataset from a CSV file, preprocesses it by tokenizing text and scores, and converts it into a Dataset object.

## Training Setup: 
The SFTTrainer is initialized with the pre-trained model, training dataset, training arguments, and specific configurations such as peft_config and formatting_func. It prepares the model for training with settings like batch size, learning rate, and optimization algorithm.

## Model Training: 
The trainer.train() function initiates the training process.

## Model Saving: 
After training, the model is saved to a specified directory.

## Text Processing Functions: 
Two utility functions, truncate_txt and gen_prompt, are defined for text truncation and prompt generation, respectively.

## Inference: 
In the inference section, it reads another dataset for testing, iterates over each essay, generates prompts, and utilizes the model to predict scores. The scores are parsed from the generated output. There's also a timeout mechanism to prevent lengthy computations.

## Output Generation: 
The predicted scores are collected and formatted into a DataFrame, which is then saved as a CSV file for further analysis.

In [ ]:
!pip install /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl

In [ ]:
!pip install accelerate

In [ ]:
!pip install /kaggle/input/llm-peft-pkg/peft-0.10.0-py3-none-any.whl

In [ ]:
from accelerate import Accelerator
import transformers
import bitsandbytes
import torch
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from peft import LoraConfig, get_peft_model, TaskType, AutoPeftModelForCausalLM

In [ ]:
accelerator = Accelerator()

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
MODEL_PATH = "/kaggle/input/gemma/transformers/1.1-7b-it/1"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_PATH,
#     device_map = "auto",
#     trust_remote_code = True,
#     quantization_config=quantization_config,
# )
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/peft_gemma/transformers/1.0/1", quantization_config=quantization_config, device_map = "auto", trust_remote_code = True,)

model = accelerator.prepare(model)

In [ ]:
model

In [ ]:
# lora_config = LoraConfig(
#     r=32, # Rank
#     lora_alpha=32,
#     target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type=TaskType.CAUSAL_LM 
# )

In [ ]:
# def print_number_of_trainable_model_parameters(model):
#     trainable_model_params = 0
#     all_model_params = 0
#     for _, param in model.named_parameters():
#         all_model_params += param.numel()
#         if param.requires_grad:
#             trainable_model_params += param.numel()
#     return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

# print(print_number_of_trainable_model_parameters(model))

In [ ]:
# peft_model = get_peft_model(model, 
#                             lora_config)
# print(print_number_of_trainable_model_parameters(peft_model))

In [ ]:
# TRAIN_DF_FILE = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv"
# df = pd.read_csv(TRAIN_DF_FILE)
# df["score"] = df["score"].astype(str)

In [ ]:
# from datasets import load_dataset,Dataset

# data = Dataset.from_pandas(df)
# data = data.map(lambda samples: tokenizer(samples["full_text"]), batched=True)
# data = data.map(lambda samples: tokenizer(samples["score"]), batched=True)

In [ ]:
# !pip install /kaggle/input/llm-peft-pkg/shtab-1.7.1-py3-none-any.whl
# !pip install /kaggle/input/llm-peft-pkg/tyro-0.7.3-py3-none-any.whl
# !pip install /kaggle/input/llm-peft-pkg/trl-0.8.1-py3-none-any.whl

In [ ]:
# from trl import SFTTrainer

In [ ]:
# def formatting_func(example):
#     text = f"Essay:\n{example['full_text'][0]}\n\nInstruction:\n You are a teacher. Score the given essay written by your student, the score must be an integer between 1 and 6. Your score should reflect the overall quality of the essay, considering factors such as clarity, coherence, organization, argumentation, language proficiency, and adherence to standard essay conventions. \n\nResponse: \n{example['score'][0]}"
#     return [text]

# trainer = SFTTrainer(
#     model=peft_model,
#     train_dataset=data,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=1,
#         gradient_accumulation_steps=4,
#         warmup_steps=2,
#         max_steps=10,
#         learning_rate=1e-4,
#         fp16=True,
#         logging_steps=1,
#         output_dir="outputs",
#         optim="paged_adamw_8bit"
#     ),
#     peft_config=lora_config,
#     formatting_func=formatting_func,
# )
# trainer.train()

In [ ]:
#peft_model.save_pretrained("/kaggle/working/model")

In [ ]:
def truncate_txt(text, length):
    text_list = text.split()
    
    if len(text_list) <= length:
        return text
    
    return " ".join(text_list[:length])


def gen_prompt(essay):
    
    # Truncate the texts to first 200 words for now
    #og_text = truncate_txt(essay, 150)
    return f"""   
    Essay:
    \"""{essay}\"""
    
    Instruction:
    You are a teacher. Fairly score the given essay written by your student, the score must be an integer between 1 and 6. Your score should reflect the overall quality of the essay, considering factors such as clarity, coherence, organization, argumentation, language proficiency, and adherence to standard essay conventions. Pay attention to misspellings and slang words.
    
    Response: 
    \"""\"""
    """

In [ ]:
import datetime
from tqdm import tqdm
import gc
import re

start_time = datetime.datetime.now()

In [ ]:
def parse(text):
    answer = []
    start = False
    end = False
    for l in reversed(list(text)):
        if l in '0123456789' and not end:
            start = True
            answer.append(l)
        else:
            if start:
                end = True
        
    answer = reversed(answer)
    return ''.join(answer)

In [ ]:
device = accelerator.device

tdf = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
pbar = tqdm(total=tdf.shape[0])
it = iter(tdf.iterrows())
idx, row = next(it, (None, None))

res = []

while idx is not None: 
    if (datetime.datetime.now() - start_time) > datetime.timedelta(hours=24, minutes=30):
        res.append([row["essay_id"], 0])
        idx, row = next(it, (None, None))
        pbar.update(1)
        continue
        
    torch.cuda.empty_cache()
    gc.collect()
    try:  
        messages = [
            {
                "role": "user",
                "content": gen_prompt(row["full_text"])
            }
        ]
        encoded_input = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(device)

        with torch.no_grad():
            encoded_output = model.generate(encoded_input, temperature = 0.1, max_new_tokens=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)

        decoded_output = tokenizer.batch_decode(encoded_output, skip_special_tokens=True)[0]
        mark = parse(decoded_output)
        #decoded_output = result = re.sub(r"[\s\S]*\[\/INST\]", '', decoded_output, 1)
        print(decoded_output)        
        res.append([row["essay_id"], int(mark)])
                            
    except Exception as e:
        print(f"ERROR: {e}")
        res.append([row["essay_id"], 0])
        
    finally:
        idx, row = next(it, (None, None))
        pbar.update(1)

        
pbar.close()

In [ ]:
res

In [ ]:
sub = pd.DataFrame(res, columns=['essay_id', 'score'])

sub.to_csv("submission.csv", index=False)